In [2]:
import re, requests
import pandas as pd
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Chrome('C:/Users/ysh/Downloads/chromedriver')
driver.implicitly_wait(5)

In [34]:
link_list = []

for i in range(92) : 
        
    webpage = driver.get("https://terms.naver.com/list.nhn?cid=51007&categoryId=51007&so=date.dsc&viewType=&categoryType=&page="+str(i))
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    for element in soup.find_all("strong", class_= "title"):
        for link in element.find_all("a") :
            link_list.append(link.get("href"))

In [35]:
for i in range(len(link_list)) :
    link_list.remove('#')

ValueError: list.remove(x): x not in list

In [36]:
for i in range(len(link_list)) :
    link_list.remove('/tlist')

ValueError: list.remove(x): x not in list

In [37]:
link_list

['/entry.nhn?docId=5749227&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749226&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749225&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749224&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749223&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749222&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749221&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749220&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749219&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749218&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749217&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749216&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749215&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749214&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749213&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749227&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749226&cid=51007&categoryId=51007',
 '/entry.nhn?docId=5749225&cid=51007&categoryId=

In [38]:
len(link_list)

1379

In [39]:
#데이터 베이스 구축용 데이터 프레임 
#                                     질병 , 과목       정의          원인     증상        진단/검사     치료       경과/합병증   예방방법    
disease_df = pd.DataFrame(columns=['title', 'subject', 'definition', 'cause', 'symptoms', 'diagnosis', 'treatment','progress','prevention'])

In [40]:
a=0

for i in link_list : 
    disease = [' ',' ',' ',' ',' ',' ',' ',' ',' ']
    subject = []    
    webpage = driver.get("https://terms.naver.com"+i)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    if  soup.find(string=re.compile('시술방법'))=='시술방법'  or soup.find(string=re.compile('준비사항')) == '준비사항' :         
        continue
    else :
        list = []
        
        disease[0] = soup.title.get_text() 
        subject = soup.find('table', class_='tmp_profile_tb').td.get_text(strip=True)
        disease[1] = re.sub('[\{\}\[\]\/?.;: |\)*~`!^\-_+<>@\#$%&\\\=\(\'\n"]','', str(subject))
        
        for i in range(len(soup.select(".stress"))):
            if soup.select(".stress")[i].get_text() == "정의":
                disease[2] = (soup.select(".txt")[i].get_text(strip=True))
            elif soup.select(".stress")[i].get_text() == "원인":
                disease[3] = (soup.select(".txt")[i].get_text(strip=True))
            elif soup.select(".stress")[i].get_text() == "증상":
                disease[4] = (soup.select(".txt")[i].get_text(strip=True))    
            elif soup.select(".stress")[i].get_text() == "진단/검사":
                disease[5] = (soup.select(".txt")[i].get_text(strip=True))
            elif soup.select(".stress")[i].get_text() == "치료":
                disease[6] = (soup.select(".txt")[i].get_text(strip=True))
            elif soup.select(".stress")[i].get_text() == "경과/합병증" :
                disease[7] = (soup.select(".txt")[i].get_text(strip=True))
            elif soup.select(".stress")[i].get_text() == "예방방법" :
                disease[8] = (soup.select(".txt")[i].get_text(strip=True))
                              
    disease_df.loc[a]=disease
        
    a = disease_df.index.max()+1 

In [41]:
disease_df

,title,subject,definition,cause,symptoms,diagnosis,treatment,progress,prevention
0,내측 측부인대 파열,정형외과,내측측부인대 손상은 슬관절에 과도한 외반력이 가해질 때 생기는 것으로 단독 손상으로...,슬관절에 가해지는 과도한 외반력이 가장 흔한 발생 원인이다. 손상 당시 슬관절은 약...,,손상 정도를 분류하는 여러 분류법이 있으며 보통 슬관절에 외반스트레스를 가했을 때 ...,"비스테로이드 항염증제(NSAIDs), 부목 고정 등 비수술적 치료에 반응이 좋은 편...",,
1,장경인대 증후군,정형외과,장경인대는 운동 선수들이 흔히 통증을 호소하는 부위로 무릎 장경인대와 대퇴 외과 사...,"운동 습관에 따라 갑작스러운 과도한 강도의 운동, 반복적으로 오랜 시간 가부좌 자세...",장경인대는 육상선수나 싸이클 선수 또는 다른 운동 선수들이 흔히 통증을 호소하는 부...,,50-90%의 환자에서 4-8주간의 비수술적인 치료에 호전을 보인다. 비수술적 치료...,,
2,슬개골 탈구,정형외과,슬개골은 대퇴골과 접촉하며 관절면을 이루는데 여러 가지 원인에 의해 대퇴골에서 탈구...,슬개골은 대퇴골과 접촉하며 관절면을 이루는데 여러가지 원인에 의해 대퇴골에서 탈구되...,,,갑작스럽게 외상으로 인해 최초로 탈구된 경우에는 주로 비수술적인 치료를 한다. 무릎...,,
3,화농성 관절염,정형외과,감염성관절염이라고도 알려진 화농성 관절염은 염증을 일으키는 감염원이 관절내에 들어옴...,"원인에는 세균, 바이러스, 균류 및 기생충이 포함된다. 위험 요인으로는 인공 관절,...",감염성관절염이라고도 알려진 화농성 관절염은 염증을 일으키는 감염원이 관절내에 들어옴...,,"초기치료는 일반적으로 세파졸린(cefazolin), 세프트리악손(ceftriaxon...",,
4,혈관종,성형외과,발생 단계에 있는 미분화한 모세혈관그물이 정상 혈관으로 발전하지 못해서 생긴 것을 ...,"정확한 원인은 밝혀져 있지 않으나 유전적 요인과 호르몬, 출생시 저체중 등 여러 요...","혈관종은 생후 8-12개월 동안 빠르게 자라다가(증식기, proliferating ...",대부분의 혈관종은 병력(history)과 진찰로써 정확히 진단된다. 목이나 몸통에 ...,대부분의 혈관종은 어떠한 치료도 요구되지 않는다. 일반적으로 합병증을 일으키지 않은...,,
...,...,...,...,...,...,...,...,...,...
1072,노로 바이러스,"감염내과,소화기내과,소아청소년과",노로 바이러스 감염증은 노로 바이러스에 의한 유행성 바이러스성 위장염이다. 노로 바...,칼리시 바이러스과에 속하는 노로 바이러스에 의해 발생한다. 바이러스 입자는 크기가 ...,사람이 노로 바이러스에 감염되면 평균 12-48시간의 잠복기를 거친 뒤에 갑자기 오...,"전염성이 매우 높은 바이러스성 감염 질환으로 요양원, 캠프, 군대, 지역 사회에서 ...",노로 바이러스 장염은 특별한 치료 없이 저절로 회복된다. 노로 바이러스에 특수한 항...,대부분 치료를 하지 않아도 자연적으로 치유된다. 심한 탈수나 전해질 불균형이 발생할...,노로 바이러스 감염을 예방하기 위해서는 철저한 손위생이 가장 중요하다. 흐르는 물에...
1073,한센병,"감염내과,피부과",나균에 의해 감염되는 만성 전염성 질환을 한센병이라고 합니다. 6세기에 처음 발견된...,나균이 원인 병원체입니다. 이는 가족 내에서 장기간의 긴밀한 접촉으로 인해 전파되는...,잠복기간은 9개월~20년으로 다양합니다. 한센병은 피부에 나타나는 병적인 변화의 종...,"피부에 나타나는 변화를 확인하고, 말초신경이 침범되었을 때 나타나는 증상인 과다 감...",한센병의 치료에는 여러 종류의 항생제를 함께 쓰는 병합요법이 사용됩니다. 나균에 효...,치료하지 않으면 신경계의 합병증으로 인해 사지의 무감각과 근육의 병적인 증상이 발생...,
1074,광견병,"감염내과,신경과",광견병은 광견병 바이러스(rabies virus)를 가지고 있는 동물에게 사람이 물...,리사 바이러스(Lyssavirus) 속에 속하는 광견병 바이러스가 광견병의 원인 바...,광견병 바이러스에 노출된 후 증상이 나타나는 데 걸리는 시간은 여러 가지 요인에 영...,동물에 물린 직후에 체내에 광견병 바이러스가 들어왔는지를 확인할 수 있는 검사법은 ...,,인공호흡기를 사용하면 단기간 동안 생명을 연장시킬 수 있지만 합병증이 발생하면서 결...,1) 광견병 유행지역을 여행할 때는 개를 비롯한 광견병 위험 동물과의 접촉에 주의한...
1075,감기,"가정의학과,감염내과,호흡기내과,소아청소년과,이비인후과","감기는 바이러스에 의해 코와 목 부분을 포함한 상부 호흡기계의 감염 증상으로, 사람...",200여개 이상의 서로 다른 종류의 바이러스가 감기를 일으킨다. 그 중 30~50%...,감기 바이러스에 노출된 지 1~3일 후에 증상이 나타난다. 증상은 감기 바이러스가 ...,병력 조사와 함께 임상 증상을 관찰하여 진단할 수 있다. 감기의 경우 확진 검사는 ...,특이적인 치료법은 없다.세균에 의한 2차 감염을 방지할 목적으로 항생제를 일률적으로...,"급성 중이염, 부비동염(축농증), 폐렴 등이 동반될 수 있다. 어른의 경우, 상부 ...",


In [42]:
disease_df.to_csv('naver_disease2.csv', index=False, mode='a', encoding='utf-8-sig')

In [3]:
webpage = driver.get("http://www.koicd.kr/2016/easyFind/v7.do")
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [83]:
soup.find_all('table', class_='idx_table')

[<table class="idx_table"><thead><tr><th class="col-sm-2">코드</th><th class="col-sm-4">한글</th><th class="col-sm-6">영문</th></tr></thead><tbody><tr class=""><td class="">A00-B99</td><td class="">I. 특정 감염성 및 기생충성 질환</td><td class="">Certain infectious and parasitic diseases, A00-B99</td></tr><tr class=""><td class="">A00-A09</td><td class="">장감염질환</td><td class="">Intestinal infectious diseases</td></tr><tr class=""><td class="">A00</td><td class="">콜레라</td><td class="">Cholera</td></tr><tr class=""><td class="">A00.0</td><td class="">비브리오 콜레라 01 콜레라형균에 의한 콜레라</td><td class="">Cholera due to Vibrio cholerae 01, biovar cholerae</td></tr><tr class=""><td class="">A00.1</td><td class="">비브리오 콜레라 01 엘토르형균에 의한 콜레라</td><td class="">Cholera due to Vibrio cholerae 01, biovar eltor</td></tr><tr class=""><td class="">A00.9</td><td class="">상세불명의 콜레라</td><td class="">Cholera, unspecified</td></tr><tr class=""><td class="">A01</td><td class="">장티푸스 및 파라티푸스</td><td class="">Typhoid and paratyphoid feve

In [46]:
for tr in soup.find_all('tr'):      
    print(list(tr.find_all('td')))

[<td style="text-align:center;"><img src="/images/loading.gif"/><br/>loading...</td>]
[<td class="top_memo"><a href="javascript:void(0)" onclick="javascript:kcd_memo_list(1,5,'MEMO','')">메모장</a><img alt="help" onclick="showcustomPopup('memo', event);" src="/images/2016/icon_help.png"/></td>, <td><a href="/2016/board/srvImpinsert.do?board_id=2">개선의견</a></td>, <td><a href="/2016/user/viewLogin.do">로그인</a></td>, <td><a href="/2016/user/join.do">회원가입</a></td>]
[<td class="search_label">통합검색</td>, <td class="search_input"><input class="total_search_text" id="totalSearchWrd" name="totalSearchWrd" placeholder="질병코드, 한글, 영문명을 입력하세요." type="text"/>
<img alt="검색" class="total_search_btn" src="/images/ic_search.png" style=""/></td>, <td class="search_label">인기검색어</td>, <td class="search_ticker"><div class="top_search_ticker"><div id="top_search_ticker"></div></div></td>]
[<td><input class="search_text" id="subSearchWrd" name="subSearchWrd" placeholder="검색어를 입력하세요." type="text"/></td>, <td><div cl

In [82]:
soup.select("table > tbody")

[<tbody><tr><td style="text-align:center;"><img src="/images/loading.gif"/><br/>loading...</td></tr>
 </tbody>,
 <tbody><tr>
 <td class="top_memo"><a href="javascript:void(0)" onclick="javascript:kcd_memo_list(1,5,'MEMO','')">메모장</a><img alt="help" onclick="showcustomPopup('memo', event);" src="/images/2016/icon_help.png"/></td>
 <td><a href="/2016/board/srvImpinsert.do?board_id=2">개선의견</a></td>
 <td><a href="/2016/user/viewLogin.do">로그인</a></td>
 <td><a href="/2016/user/join.do">회원가입</a></td>
 </tr></tbody>,
 <tbody><tr>
 <td class="search_label">통합검색</td>
 <td class="search_input"><input class="total_search_text" id="totalSearchWrd" name="totalSearchWrd" placeholder="질병코드, 한글, 영문명을 입력하세요." type="text"/>
 <img alt="검색" class="total_search_btn" src="/images/ic_search.png" style=""/></td>
 <td class="search_label">인기검색어</td>
 <td class="search_ticker"><div class="top_search_ticker"><div id="top_search_ticker"></div></div></td>
 </tr></tbody>,
 <tbody><tr>
 <td><input class="search_text"